# Case study
## Task
Predict income and expenses for a holdout sample of ~10K users for the month of August based on a training sample of ~10K users from Februrary through July.

Based on your judgement of the usefulness of the results, either aggregate the data into incoming & outgoing flows, or predict based on the transaction type/category.


In [2]:
# library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.neural_network import MLPClassifier
import joblib
import os

# custom modules
from LRmodel import LogisticRegressionModels
import plotter
from preprocessing import load_and_preprocess


### Loading the dataset

In [2]:
'''
data
'''
# data_train = pd.read_csv('./data/2016-09-19_79351_training.csv')
data_train = pd.read_csv("./data/2016-09-19_79351_training_feb_june.csv")
data_mcc_group = pd.read_csv('./data/mcc_group_definition.csv')
data_transaction = pd.read_csv('./data/transaction_types.csv')


# Data wrangling

In [3]:
data_train_main = data_train.copy()

In [ ]:
data_train_main.head()

In [ ]:
data_train_main.dataset_transaction.unique(), data_train_main.dataset_user.unique()

In [ ]:
data_train_main.info()

The features `dataset_transaction` and  `dataset_user` contains only one unique entry which are irrelevant. We simply drop them. 

In [7]:
data_train_main = data_train_main.drop(['dataset_transaction', 'dataset_user'], axis=1)

In [ ]:
data_train_main.head(2)

## To do
 - create ```direction``` (i.e. in/out) feature: the ```in``` and ```out``` represents the income and expenses, respectively. ```in``` denotes money coming into the user acccount and ```out``` denotes money going out of the user account.

 - creating month/day feature from the transaction date.

 - fill NAs in ```mcc_group``` with mode and miscellaneous. We will fill the NAs locally per user. Specifically, we will take the ```mcc_group``` of a user and check for the most frequent transaction type and use this mode to replace any NA for such user. In the event there is no mode, we will simply categorize this as miscellaneous (i.e. 16) since no modal value.

In [ ]:
'''
filtered data_transaction based in transaction_type
'''

# filtering data_transaction.type

# data_train_main.user_id.nunique()
selected_types = list(data_train_main.transaction_type.unique()); selected_types
data_transaction_filtered = data_transaction[data_transaction.type.isin(selected_types)]

data_transaction_filtered.reset_index(drop = True, inplace = True); data_transaction_filtered

In [ ]:
# Grouping the transaction type into in and out
type_in = data_transaction_filtered[data_transaction_filtered.direction == 'In'].type
type_out = data_transaction_filtered[data_transaction_filtered.direction == 'Out'].type
type_in, type_out = list(type_in), list(type_out); 
type_in, type_out

In [15]:
## adding in/out feature to data_train_main

data_train_main['direction'] = data_train_main.transaction_type.apply(lambda x: 'In' if x in type_in else 'Out')

In [16]:
# mapping and adding agent feature to data_train_main

type_to_agent = {
    'BBU': 'Partner',
    'CT': 'Bank Account',
    'DR': 'Bank Account',
    'PF': 'Card',
    'PT': 'Card',
    'BUB': 'Partner',
    'DD': 'Bank Account',
    'DT': 'Bank Account',
    'FT': 'Bank Account',
    'TUB': 'Partner'
}

data_train_main['agent'] = data_train_main.transaction_type.apply(lambda x: type_to_agent[x])

In [ ]:
# A quick check on the data_train_main
data_train_main[data_train_main.transaction_type == 'PT']

=====================================

In [ ]:
# extract day/month from data_train_main.transaction_date
data_train_main['transaction_date'] = pd.to_datetime(data_train_main.transaction_date)
data_train_main['day'] = data_train_main.transaction_date.dt.day
data_train_main['month'] = data_train_main.transaction_date.dt.month

### Filling in the missing values in mcc_group feature

=====================================

![title](./img/na_fill.png)


As shown above, we filled the NAs locally per user. Specifically, we take the ```mcc_group``` of a user and check for the most frequent transaction type and use this mode to replace any NA for such user. In the event there is no mode, we simply categorize this as miscellaneous (i.e. 16) since no modal value.

In [24]:
# Function to fill missing values with mode or miscellaneous
def fill_missing_with_mode(df, group_col, target_col):
    
    def mode_function(x):
        modes = x.mode()
        if not modes.empty:
            return modes[0]
        else:
            return 16 #miscellaneous
    
    df[target_col] = df.groupby(group_col)[target_col].transform(lambda x: x.fillna(mode_function(x)))
    
    return df


data_train_main_filled = fill_missing_with_mode(data_train_main, 'user_id', 'mcc_group')


In [ ]:
# Testing the result for a user
[data_train_main_filled['user_id']=='001f9baedaf3c8487c344d25b0eda9fc']

In [ ]:
# Distribution of mcc_group before filling NAs

plt.figure(figsize=(15, 6))
data_train_main.mcc_group.value_counts().plot(kind = 'bar')

In [ ]:
# Distribution of mcc_group after fill

plt.figure(figsize=(15, 6))
data_train_main_filled.mcc_group.value_counts().plot(kind = 'bar')

# checking distribution of response feature

In [ ]:
# Distribution of direction
data_train_main_filled.direction.value_counts().plot(kind = 'bar')

The plot shows that the target variable `direction` is imbalanced. With income accounting for 18% and expenses 82%. 

In [ ]:
#percentage distribution of direction
data_train_main_filled.direction.value_counts(normalize=True) * 100

## Feature engineering

In [ ]:
# encoding categorical columns

categorical_cols = data_train_main_filled.select_dtypes(include = 'object').columns
print(f'categorical_cols:{categorical_cols}')
for col in categorical_cols[1:]:
    data_train_main_filled[col] = LabelEncoder().fit_transform(data_train_main_filled[col])



# set mcc_group into integer

data_train_main_filled['mcc_group'] = data_train_main_filled['mcc_group'].astype(int)

In [ ]:
# A view of the updated data
data_train_main_filled.head()

In [37]:
# Dropping transaction_date, transaction_type, user_id
data_train_main_filled.drop(['user_id', 'transaction_date', 'transaction_type'], axis=1, inplace=True)


In [ ]:
# A quick correlation check
data_train_main_filled.corr()

## splitting

In [41]:
'''
split data
'''
X = data_train_main_filled.drop('direction', axis=1)
y = data_train_main_filled['direction']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


In [ ]:
# Checking the proportion of the classes in the train, validation and test sets
y_train.value_counts(normalize=True) * 100, y_val.value_counts(normalize=True) * 100, y_test.value_counts(normalize=True) * 100

In [45]:
'''
grids search on RF
'''

run_grid_search = False
if run_grid_search:
    
    param_grid = {
        'class_weight': [{0: 1.3, 1: 1.0},{0: 1.4, 1: 1.0}, {0: 1.45, 1: 1.0}, {0: 1.5, 1: 1.0}],
        # 'class_weight': [{0: 2.8, 1: 0.6}, {0: 2.4, 1: 0.6},  {0: 2.2, 1: 0.6}],
        'n_estimators': [50, 100],
        'max_depth': [None, 10, 20],
    }
    

    grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, scoring='roc_auc', cv=3, verbose=3)
   
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_;best_params
    print("Best parameters:", best_params)

# Methods

![title](./img/lr.png)
## 1. LR (logistic regression): 
we use the LR methods as baseline and in addition, we add penalties to the cost function to balance the target variable.

## 2. RF (random forest)

![title](img/rfTree.png)
`(image source: https://upload.wikimedia.org/wikipedia/commons/4/4e/Random_forest_explain.png)`

In classification problem, the RF method works by constructing multiple decision trees during training and returns the majority vote from these trees.

baseline RF assumes equal importance for all classes, and the weighted approach introduces class weights to modify the splitting criteria of the labels to prioritize minority class.

## 3. MLP (multilayer perceptron)
mlp is a feedforward neural network consisting of fully connected neurons with nonlinear activation functions 





paramter:

- `class_weight`: in the weighted RF, changes to `class_weight` will modify the outcome of the model. after some experiments, the class weight is set as `class_weight={0:1.45, 1:1.0}`
- `max_depth`: the maximum depth of the tree
- `n_estimator`: higher (lower) `n_estimator` could increase (reduce) model accuracy
- weights on LR: to tune the model sensitivity to imbalanced response feature, use `weight_positive=1.` and  `weight_negative=1.45`. these values are thesame as in `class_weight`
- `hidden_layer_sizes`: The ith element represents the number of neurons in the ith hidden layer.
- `activation`: activation function for the hidden layer. other options are {identity, logistic, tanh}.
- `solver`: the solver for weight optimization. other options to try are `lbfgs` and `sgd`.

In [ ]:
'''
train model
'''

n_estimator = 100
seedS = 42
lr = 0.01
max_iter = 1000
max_depth = 15

models = {
    "Baseline RF": RandomForestClassifier(n_estimators=n_estimator, random_state=seedS),
    "Weighted RF": RandomForestClassifier(n_estimators=n_estimator, class_weight={0:1.45, 1:1.0}, max_depth=max_depth, random_state=seedS),
    "Baseline LR": LogisticRegressionModels(model_type="baseline", learning_rate=lr, max_iter=max_iter),
    "cost LR": LogisticRegressionModels(model_type="cost_sensitive", weight_positive=1., weight_negative=1.45, learning_rate=lr, max_iter=max_iter),
    "MLP": MLPClassifier(hidden_layer_sizes=(50,), activation='relu', solver='adam', verbose=False,learning_rate_init=lr, max_iter=max_iter, random_state=seedS),
} #relu

results = {}
for name, model in models.items():
    print(f"Training {name}")
    if model.__class__.__name__ in ['RandomForestClassifier', 'MLPClassifier']:
        model.fit(X_train, y_train)
    elif model.__class__.__name__ == 'LogisticRegressionModels':
        model.fit(X_train, y_train, X_val, y_val)
    else:
        raise ValueError("Model not supported")
        
    show_curves = False
    if show_curves:
        if isinstance(model, (LogisticRegressionModels)):  
            plt.figure(figsize=(10, 6))
            plt.plot(model.train_losses, label='Training Loss')
            plt.plot(model.val_losses, label='Validation Loss')
            plt.xlabel('Iteration')
            plt.ylabel('Loss')
            plt.title(f'{name} Training and Validation Loss Curves')
            plt.legend()
            plt.grid()
            plt.show()

    # pred
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    

    # metric
    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    f1 = f1_score(y_test, y_pred_test)
    
    if model.__class__.__name__ in ['RandomForestClassifier', 'MLPClassifier']:
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_pred_proba_test)
    elif model.__class__.__name__ == 'LogisticRegressionModels':
        y_pred_proba_test = model.predict_proba(X_test)
        auc = roc_auc_score(y_test, y_pred_proba_test)

    # Save model
     # joblib.dump(model, f"models/{name.replace(' ', '_')}_model.pkl")
    
    results[name] = {
        "Train Accuracy": train_accuracy,
        "Val Accuracy": val_accuracy,
        "Test Accuracy": test_accuracy,
        "AUC": auc,
        "f1": f1,
        "Model": model,
        }
    print(f"--- {name} saved ---")
    print(f"Train Accuracy: {train_accuracy:.2f}")
    print(f"Val Accuracy: {val_accuracy:.2f}")
    print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
#Check test accuracy
methods = list(results.keys())
test_accuracies_ = [metrics['Test Accuracy'] for metrics in results.values()]

list(zip(methods,test_accuracies_))

In [ ]:
'''
plot accuraries
'''

labels = list(results.keys())
train_accuracies = [metrics['Train Accuracy'] for metrics in results.values()]
val_accuracies = [metrics['Val Accuracy'] for metrics in results.values()]
test_accuracies = [metrics['Test Accuracy'] for metrics in results.values()]

bar_width = 0.2
indices = np.arange(len(labels))
figsize = (16, 8)
plotter.plot_barplot( indices, train_accuracies, val_accuracies, test_accuracies, labels, bar_width=bar_width, figsize=figsize)


In [ ]:
# Check the F1 score
f1_ = [metrics['f1'] for metrics in results.values()]

list(zip(methods,f1_))

In [ ]:
'''
plot AUC curves
'''
from sklearn.metrics import roc_curve

def plot_auc_curves(results, X_test, y_test, figsize=(10, 6)):
    """
    ROC curves here.
    """
    plt.figure(figsize=figsize)
    for name, metrics in results.items():
        model = metrics["Model"]
        if model.__class__.__name__ in ['RandomForestClassifier', 'MLPClassifier']:
            y_pred_proba_test = model.predict_proba(X_test)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba_test)
        elif model.__class__.__name__ == 'LogisticRegressionModels':
            y_pred_proba_test = model.predict_proba(X_test)
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba_test)
        else:
            continue 
        
        plt.plot(fpr, tpr, label=f'{name} (AUC = {metrics["AUC"]:.2f})')
    plt.plot([0, 1], [0, 1], "k--", label="Random Guessing")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="best", fontsize=20)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

figsize=(12, 8)
plotter.plot_auc_curves(results, X_test, y_test, figsize=figsize)

## Test case

In [11]:
import importlib
from preprocessing import load_and_preprocess
import preprocessing2 #import load_and_preprocessT

# lets compare `July_data` with `predict_df`

In [38]:
#Predicted Data
output_csv_path = "./result/predictions_all_models.csv"
prediction_df_ = pd.read_csv(output_csv_path)
prediction_df_.head()

,Baseline_LR_prediction,Baseline_LR_meaning,Baseline_LR_probability,Baseline_RF_prediction,Baseline_RF_meaning,Baseline_RF_probability,Weighted_RF_prediction,Weighted_RF_meaning,Weighted_RF_probability,Cost-sensitive_LR_prediction,Cost-sensitive_LR_meaning,Cost-sensitive_LR_probability,MLP_prediction,MLP_meaning,MLP_probability
0,0,In,1.416394e-43,0,In,0.3725,0,In,0.341104,0,In,1.244572e-49,0,In,1.459896e-07
1,0,In,2.185507e-37,0,In,0.1600,0,In,0.171639,0,In,1.373175e-42,0,In,5.842439e-07
2,0,In,3.889676e-75,0,In,0.1400,0,In,0.217397,0,In,1.504343e-85,0,In,1.586080e-10
3,0,In,2.227612e-04,0,In,0.3725,0,In,0.341104,0,In,5.113552e-05,0,In,1.628183e-02
4,0,In,1.405608e-02,0,In,0.1400,0,In,0.217397,0,In,5.827044e-03,0,In,9.510073e-02


In [22]:
importlib.reload(preprocessing2)

<module 'preprocessing2' from '/Users/abdullahi/Documents/github/income/src/preprocessing2.py'>

In [34]:
# July data
new_data_dir = "../data/2016-09-19_79351_training_july.csv" 
new_data = preprocessing2.load_and_preprocessT(new_data_dir, training=False)

new_data.head(2)

original df shape: (70986, 7)
actual False


/Users/abdullahi/Documents/github/income/src/preprocessing2.py:56: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train["transaction_date"] = pd.to_datetime(df_train["transaction_date"])


categorical_cols:Index(['direction', 'agent'], dtype='object')
y:<class 'pandas.core.series.Series'>
using --training == predict--


,mcc_group,amount_n26_currency,direction,agent,day,month
0,2,202,1,0,7,1
1,1,173,1,0,7,1
